In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import seaborn as sb
import lightgbm as lgb
import tensorflow as tf
import math
import matplotlib.pyplot as plt
# from sklearn import model_selection
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing

In [2]:
train_data = pd.read_csv("train_small.csv", header=None, names=["ID_code", "target"]+["var_"+str(i) for i in range(200)])

In [3]:
train, val = train_test_split(train_data, test_size=0.1)

In [32]:
# build autoencoder
# https://blog.keras.io/building-autoencoders-in-keras.html

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=[200]))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
#model.add(tf.keras.layers.Dense(32, activation='relu'))
#model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(2, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 64)                12864     
_________________________________________________________________
dense_13 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_14 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_15 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_16 (Dense)             (None, 2)                 66        
Total params: 20,226
Trainable params: 20,226
Non-trainable params: 0
_________________________________________________________________


In [6]:
X_train = train.values[:,2:]
X_val = val.values[:,2:]
Y_train = train.values[:,1]
Y_train = Y_train.astype('int')
Y_val = val.values[:,1]
Y_val = Y_val.astype('int')

min_max_scaler = preprocessing.MinMaxScaler()
x_train_minmax = min_max_scaler.fit_transform(X_train)
x_val_minmax = min_max_scaler.fit_transform(X_val)

In [7]:
class_weight = {0: 1., 1: 10}
targets = np.array(Y_train).reshape(-1)
one_hot_targets = np.eye(2)[targets]
model.fit(x_train_minmax, one_hot_targets, batch_size=512, class_weight=class_weight, epochs=100, validation_split=0.0)

Epoch 1/100
63051/63051 [==============================] - 2s 36us/sample - loss: 1.1965
Epoch 2/100
63051/63051 [==============================] - 1s 13us/sample - loss: 0.9818
Epoch 3/100
63051/63051 [==============================] - 1s 12us/sample - loss: 0.9249
Epoch 4/100
63051/63051 [==============================] - 1s 12us/sample - loss: 0.9069
Epoch 5/100
63051/63051 [==============================] - 1s 13us/sample - loss: 0.9141
Epoch 6/100
63051/63051 [==============================] - 1s 14us/sample - loss: 0.9111
Epoch 7/100
63051/63051 [==============================] - 1s 13us/sample - loss: 0.8894
Epoch 8/100
63051/63051 [==============================] - 1s 13us/sample - loss: 0.9086
Epoch 9/100
63051/63051 [==============================] - 1s 12us/sample - loss: 0.8950
Epoch 10/100
63051/63051 [==============================] - 1s 13us/sample - loss: 0.8892
Epoch 11/100
63051/63051 [==============================] - 1s 13us/sample - loss: 0.8937
Epoch 12/100
63051/

63051/63051 [==============================] - 1s 12us/sample - loss: 0.7050
Epoch 93/100
63051/63051 [==============================] - 1s 12us/sample - loss: 0.6937
Epoch 94/100
63051/63051 [==============================] - 1s 12us/sample - loss: 0.7101
Epoch 95/100
63051/63051 [==============================] - 1s 12us/sample - loss: 0.6966
Epoch 96/100
63051/63051 [==============================] - 1s 13us/sample - loss: 0.6927
Epoch 97/100
63051/63051 [==============================] - 1s 12us/sample - loss: 0.6844
Epoch 98/100
63051/63051 [==============================] - 1s 13us/sample - loss: 0.6846
Epoch 99/100
63051/63051 [==============================] - 1s 12us/sample - loss: 0.6870
Epoch 100/100
63051/63051 [==============================] - 1s 13us/sample - loss: 0.6878


In [8]:
# get output from last hidden layer
last_hidden_output = tf.keras.backend.function([model.layers[0].input], [model.layers[-2].output])

train_pluss = train.copy()
val_pluss = val.copy()
for i, row in enumerate(np.array(last_hidden_output(x_train_minmax)).T):
    train_pluss[("var_"+str(i+200))] = row
for i, row in enumerate(np.array(last_hidden_output(x_val_minmax)).T):
    val_pluss[("var_"+str(i+200))] = row
    
print(train_pluss.head())
print(val_pluss.head())

            ID_code  target    var_0   var_1    var_2   var_3    var_4  \
53467  train_152456       0  15.2660  1.5756  17.4416  6.3997  10.8700   
19103   train_54311       0  12.3749 -1.9334  12.1989  3.4742  10.3276   
3949    train_11144       0  10.1342 -7.5167   8.4104  6.2085   9.9965   
69082  train_197138       0   7.2032 -6.2748   9.2712  7.7527  11.9160   
51398  train_146657       0  17.2041 -4.7456  13.4534  6.5172  11.6681   

         var_5   var_6    var_7  ...  var_222  var_223  var_224   var_225  \
53467  -9.4275  4.6510  23.6536  ...     -0.0     -0.0     -0.0  0.945553   
19103   3.9727  7.6310  20.2776  ...     -0.0     -0.0     -0.0  0.101307   
3949  -16.1401  6.2026  18.5395  ...     -0.0     -0.0     -0.0  2.045673   
69082 -12.3365  6.0507  10.8860  ...     -0.0     -0.0     -0.0  0.838510   
51398   3.0272  5.8702  10.0972  ...     -0.0     -0.0     -0.0 -0.000000   

        var_226   var_227  var_228   var_229  var_230  var_231  
53467 -0.000000 -0.000000  

In [9]:
# parameters are from grid search and experimentation
param = {
    'bagging_freq': 4,
    'bagging_fraction': 0.5,
    'boost_from_average':'false',
    'boost': 'gbdt',
    'feature_fraction': 0.1,
    'learning_rate': 0.01,
    'max_depth': -1,
    'metric':'auc', 
    'min_data_in_leaf': 50,
    'min_sum_hessian_in_leaf': 10,
    'num_leaves': 50,
    'min_gain_to_split': 1,
    'num_threads': 8,
    'tree_learner': 'serial',
    'objective': 'binary',
    'verbosity': 1
}

In [10]:
features = [c for c in train_pluss.columns if c not in ["ID_code", "target"]]

# using lightbm over catboost because no categorical features
lightbm_train = lgb.Dataset(train_pluss[features], label=train_pluss["target"])
lightbm_val = lgb.Dataset(val_pluss[features], label=val_pluss["target"])
clf = lgb.train(param, lightbm_train, 100000, valid_sets = [lightbm_train, lightbm_val], verbose_eval=1000, early_stopping_rounds = 3000)

Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.977118	valid_1's auc: 0.849413
[2000]	training's auc: 0.995132	valid_1's auc: 0.852779
[3000]	training's auc: 0.998837	valid_1's auc: 0.854417
[4000]	training's auc: 0.999495	valid_1's auc: 0.855253
[5000]	training's auc: 0.999678	valid_1's auc: 0.856031
[6000]	training's auc: 0.999772	valid_1's auc: 0.856627
[7000]	training's auc: 0.999822	valid_1's auc: 0.856579
[8000]	training's auc: 0.999854	valid_1's auc: 0.856597
[9000]	training's auc: 0.999876	valid_1's auc: 0.85682
[10000]	training's auc: 0.999894	valid_1's auc: 0.857106
Did not meet early stopping. Best iteration is:
[9998]	training's auc: 0.999894	valid_1's auc: 0.857106


In [25]:
# print some information to get an idea of how well it worked
train_predict = clf.predict(train_pluss[features], num_iteration=clf.best_iteration)
train_predict_binary = list(map(lambda x : math.floor(x+0.5), train_predict))
print(roc_auc_score(Y_train, train_predict_binary))

val_predict = clf.predict(val_pluss[features], num_iteration=clf.best_iteration)
val_predict_binary = list(map(lambda x : math.floor(x+0.5), val_predict))
print(roc_auc_score(Y_val, val_predict_binary))

0.9931852355368357
0.7184577821714264


In [26]:
test_data = pd.read_csv("test_small.csv")

In [30]:
X_test = test_data.values[:,1:]
x_test_minmax = min_max_scaler.fit_transform(X_test)

test_pluss = test_data.copy()
for i, row in enumerate(np.array(last_hidden_output(x_test_minmax)).T):
    test_pluss[("var_"+str(i+200))] = row
print(test_pluss.head())

    ID_code    var_0   var_1    var_2   var_3    var_4   var_5   var_6  \
0   train_0   8.9255 -6.7863  11.9081  5.0930  11.4607 -9.2834  5.1187   
1   train_3  11.0604 -2.1518   8.9522  7.1957  12.5846 -1.8361  5.8428   
2   train_7  13.5580 -7.9881  13.8776  7.5985   8.6543  0.8310  5.6890   
3  train_12   8.7671 -4.6154   9.7242  7.4242   9.0254  1.4247  6.2815   
4  train_14  13.8080  5.0514  17.2611  8.5120  12.8517 -9.1622  5.7327   

     var_7   var_8  ...  var_222  var_223  var_224   var_225   var_226  \
0  18.6266 -4.9200  ...     -0.0     -0.0     -0.0  0.667717 -0.000000   
1  14.9250 -5.8609  ...     -0.0     -0.0     -0.0 -0.000000  0.489251   
2  22.3262  5.0647  ...     -0.0     -0.0     -0.0  0.589201 -0.000000   
3  12.3143  5.6964  ...     -0.0     -0.0     -0.0  1.935561 -0.000000   
4  21.0517 -4.5117  ...     -0.0     -0.0     -0.0  0.987471 -0.000000   

    var_227  var_228   var_229  var_230  var_231  
0  0.071230     -0.0  0.585236     -0.0     -0.0  
1  0.617

In [31]:
test_predict = clf.predict(test_pluss[features], num_iteration=clf.best_iteration)
test_predict_binary = list(map(lambda x : math.floor(x+0.5), test_predict))
print(sum(test_predict_binary)/len(test_predict_binary))

0.06639548990223364


In [ ]:
# save test predictions to csv file with no headers
submission_df = pd.DataFrame(test_predict_binary)
submission_df.to_csv("submission.csv", encoding='utf-8', index=False)